In [1]:
def objectiveXGB(params):
    params = {'eta': params['eta'],
              'max_depth': int(params['max_depth']),
              'learning_rate': params['learning_rate'],
              'alpha': params['alpha'],
              'n_estimators': int(params['n_estimators'])}

    clf = xgb.XGBRegressor(random_state=random_state, **params)

    kf = KFold(n_splits=folds, shuffle=shuffle)
    loss_list = []

    for train_index, test_index in kf.split(X_train):
        X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]

        clf_fold = clone(clf)
        clf_fold.fit(X_train_fold, y_train_fold, early_stopping_rounds=25,
                     eval_set=[(X_test_fold, y_test_fold)], verbose=False)

        prediction = clf_fold.predict(X_test_fold)
        loss = rmsle(prediction, y_test_fold)
        loss_list.append(loss)

    score = np.mean(loss_list)

    return score

In [2]:
space={
    'max_depth' : hp.quniform('max_depth', 2, 30, 1),
    'alpha': hp.loguniform('alpha', -5, 0),
    'learning_rate': hp.loguniform('learning_rate', -5, 0),
    'eta': hp.loguniform('eta', -5, 0),
    'n_estimators': hp.quniform('n_estimators', 100, 10000, 1),
}

best=fmin(fn=objectiveXGB, # function to optimize
          space=space,
          algo=tpe.suggest, # optimization algorithm, hyperotp will select its parameters automatically
          max_evals=100, # maximum number of iterations
          #trials=trials, # logging
          rstate = np.random.default_rng(random_state) # fixing random state for the reproducibility
          )

print("best {}".format(best))

NameError: name 'hp' is not defined